In [1]:
import numpy as np
import pandas as pd
import random
import torch
import pickle
from datetime import datetime

In [2]:
import mlflow

In [3]:
from PortfolioConstructor import PortfolioConstructor
from ExchnageEnv import MarketEnvironment

In [4]:
if torch.cuda.is_available():
    torch.set_default_device('cuda') 
    torch.get_default_device()
    device = 'cuda'
    
else:
    device = 'cpu'

print(f"device : {device}")

device : cuda


In [5]:
tid = datetime.now().strftime("%Y_%m_%d_%H_%M")

In [6]:
with open("/home/naradaw/dev/Charles_Schwab/data/symbol_universe/snp_unique_100_2019", "rb") as fp:
    symbol_universe = pickle.load(fp)
    
symbol_universe = symbol_universe[:20]

In [7]:
feature_set_path = "/home/naradaw/dev/Charles_Schwab/data/w_features/v1/2024_10_31/feature_set_2024_10_31_11_18.pkl"

with open(feature_set_path, 'rb') as f:
    feature_set = pickle.load(f)

In [8]:
data_path = "/home/naradaw/dev/Charles_Schwab/data/w_features/v1/2024_10_31/dataset_sqs_2024_10_31_11_18.pkl"

In [9]:
mlflow_tracking_uri = 'file:/home/naradaw/dev/Charles_Schwab/code/RnD/v4/mlflow_experiments'
mlflow.set_tracking_uri(mlflow_tracking_uri)

experiment_name = "/portfolio-contructor-v4"
mlflow.set_experiment(experiment_name)

2024/11/11 15:30:15 INFO mlflow.tracking.fluent: Experiment with name '/portfolio-contructor-v4' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/naradaw/dev/Charles_Schwab/code/RnD/v4/mlflow_experiments/589106785306301247', creation_time=1731319215217, experiment_id='589106785306301247', last_update_time=1731319215217, lifecycle_stage='active', name='/portfolio-contructor-v4', tags={}>

# Experiment params

In [10]:
episodes = 1000
eval_step = 1
train_step = 1

learning_rate = 0.005

symbol_universe= symbol_universe
num_features= len(feature_set)
d_model = 88
nheads = 1
num_transformer_layers = 2

episode_duration= 12   
holding_period = 1
train_test_split= 0.7
symbol_universe = symbol_universe
feature_set= feature_set

In [11]:
# symbol_universe = random.choices(symbol_universe, k = 20)
# symbol_universe

# Utility

In [12]:
''' 
sharpe ratio measures the excess return of the portfolio over the 
volatility of it -> risk adjusted performance
'''

def sharp_ratio_(rewards, tran_costs):

	# rewards = [r.detach().cpu().numpy() for r in rewards]
	mean = sum(rewards) / len(rewards)
	At = sum(r - t for r, t in zip(rewards, tran_costs)) / len(rewards)
	vol = sum((r - mean) ** 2 for r in rewards) / len(rewards)
	vol = vol ** 0.5

	return (At - 1e-7) / (vol + 1e-9)

In [13]:
def evaluate(model, env):
    is_end = False
    rewards = []
    tran_costs = []
    
    env.reset(mode = "test")
    state = env.get_state()

    print("")
    while not is_end:
        _, allocations = model(state)
        state, reward, is_end, tran_cost = env.step(allocations)

        rewards.append(reward)
        tran_costs.append(tran_cost)

    sharp_ratio = sharp_ratio_(rewards, tran_costs)

    return sharp_ratio, model

# Train

In [14]:
# import mlflow
# mlflow.login()

In [15]:
with mlflow.start_run(run_name = f"v3_training_{tid}") as run:
    params = {
            "learning_rate": learning_rate,
            "train_step": train_step,
            "eval_step": eval_step,
            "metric_function": 'sharpe',
            "optimizer": "Adam",
            
            "symbol_universe" : symbol_universe,
            "feature_set" : feature_set,
            "d_model" : d_model,
            "nheads" : nheads,
            "num_transformer_layers" : num_transformer_layers,

            "episode_duration" : 12,    
            "holding_period" : 1,
            "train_test_split" : 0.8,
            "symbol_universe" : symbol_universe,
            "feature_set" : feature_set,

        }
    # Log training parameters.
    mlflow.log_params(params)

    portfolio_constructor = PortfolioConstructor(
        device = device,
        symbol_universe= params['symbol_universe'],
        num_features= len(params['feature_set']),
        d_model = params['d_model'],
        nheads = params['nheads'],
        num_transformer_layers = params['num_transformer_layers'],
    )

    market_env = MarketEnvironment(
        device = device,
        data_path = data_path,
        holding_period = params['holding_period'],
        episode_duration = params['episode_duration'],
        train_test_split = params['train_test_split'],
        symbol_universe = params['symbol_universe'],
        feature_set = params['feature_set']
        )

    portfolio_constructor.cuda()
    portfolio_constructor.train()
    market_env.reset(mode = "train")

    optimizer = torch.optim.Adam(portfolio_constructor.parameters(), lr = learning_rate)

    max_reward = -1

    for episode in range(episodes):
        is_end = False
        returns = []
        tran_costs = []
        nlls = []
        all_allocations = []

        market_env.reset(mode = "train", transaction_cost= 1e-7)
        state = market_env.get_state()
        
        while not is_end:
            symbol_idx, allocations = portfolio_constructor(state)
            state, return_, is_end, tran_cost = market_env.step(allocations)

            all_allocations.append(allocations)
            returns.append(return_)
            tran_costs.append(tran_cost)

        sharp_ratio = sharp_ratio_(returns, tran_costs)

        loss = -sharp_ratio

        loss.backward(retain_graph=True)

        if (episode + 1) % train_step == 0:

                    print("-------------------------------------")
                    print("training model --")
                    print('Step {}: last loss = {:.5f}\r'.format(episode, loss), end='')
                    print()
                    mlflow.log_metric("train loss", f"{loss:2f}", step=episode)

                    optimizer.step()
                    optimizer.zero_grad()
                    count = 0
                    
        if (episode + 1) % eval_step == 0:
            print("eval step --")
            with torch.no_grad():
                
                portfolio_constructor.eval()
                reward_val, portfolio_constructor = evaluate(portfolio_constructor, market_env)
                portfolio_constructor.train()

                print('Step {}: val_rewards = {}'.format(episode, reward_val))
                mlflow.log_metric("eval_sharpe", f"{reward_val:2f}", step=episode)

                if max_reward < reward_val:
                    max_reward = reward_val

                    print("*** found better model ***")
                print()
    mlflow.pytorch.log_model(portfolio_constructor, f"portfolio_constructor_{tid}")

/home/naradaw/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


-------------------------------------
training model --
Step 0: last loss = -0.12630
eval step --

Step 0: val_rewards = 0.5809145675615469
*** found better model ***

-------------------------------------
training model --
Step 1: last loss = -0.17433
eval step --

Step 1: val_rewards = 0.7313471853873924
*** found better model ***

-------------------------------------
training model --
Step 2: last loss = -0.66412
eval step --

Step 2: val_rewards = 0.7141158193210443

-------------------------------------
training model --
Step 3: last loss = -0.09478
eval step --

Step 3: val_rewards = 0.693166598948712

-------------------------------------
training model --
Step 4: last loss = -0.14319
eval step --

Step 4: val_rewards = 0.6993151464882541

-------------------------------------
training model --
Step 5: last loss = -0.09449
eval step --

Step 5: val_rewards = 0.7042044318487124

-------------------------------------
training model --
Step 6: last loss = -0.32841
eval step --

St

2024/11/11 15:34:11 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [16]:
all_allocations[-5:]

[tensor([0.1000, 0.1000, 0.0000, 0.0000, 0.1000, 0.1000, 0.1000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.1000, 0.0000, 0.1000, 0.1000, 0.1000, 0.0000,
         0.0000, 0.1000], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.1000, 0.0000, 0.1000, 0.0000, 0.1000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1000, 0.1000, 0.0000, 0.1000, 0.1000, 0.1000, 0.0000, 0.1000,
         0.1000, 0.0000], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.1000, 0.1000, 0.1000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.0000, 0.1000, 0.0000,
         0.1000, 0.0000], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.1000, 0.1000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.1000, 0.0000,
         0.0000, 0.1000, 0.1000, 0.1000, 0.1000, 0.0000, 0.1000, 0.1000, 0.0000,
         0.0000, 0.0000], device='cuda:0', grad_fn=<SoftmaxBackward0>),
 tensor([0.1000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.0000,

In [17]:
import sys

sys.exit()

SystemExit: 

/home/naradaw/miniconda3/envs/tf-wsl/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3558: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [18]:
mlflow.pytorch.log_model(
        pytorch_model=portfolio_constructor,
        artifact_path = "portfolio_constructor_{tid}",
        # input_example = market_env.get_random_state(),
        registered_model_name="portfolio-constructor-v4",
    )

2024/11/11 15:39:03 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'portfolio-constructor-v4'.
Created version '1' of model 'portfolio-constructor-v4'.
